Load the data

In [ ]:
import pandas as pd
#import matplotlib.pyplot as plt

#%matplotlib inline

path = r"C:\Users\Surface\Masterarbeit\data\Baseline.xlsx"

df = pd.read_excel(path)


Transformation Pipeline

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# get the numerical features
data_num = df.drop('x_Einzug [mm]', axis = 1)
# get the target values
data_labels = df['x_Einzug [mm]'].to_numpy()

# pipeline for preprocessing the data

# Standard Scaler for distribution with 0 mean and 1 std., normal distributed data
data_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

data_prep = data_pipeline.fit_transform(data_num)

X_train, X_test, y_train, y_test = train_test_split(data_prep, data_labels, test_size= 0.2, random_state=42)


Gradient Boosting: Decision trees die Fehler korrigieren

In [3]:
import xgboost
from sklearn.metrics import r2_score

# Initialize the XGBoost regressor
xgb_reg = xgboost.XGBRegressor()

# Fit the model to the training data
xgb_reg.fit(X_train, y_train)

#Get feature importances from the trained model
importances = xgb_reg.feature_importances_
features = ['Matrizenradius [mm ]', 'Stempelkantenradius [mm]', 'Blechdicke [mm]',
            'Platinenabmessung [mm]', '1/4 Niederhalterkraft [N]', 'Ziehtiefe [mm]']

# Create a DataFrame to display feature importance, sorted descending
xgb_feat_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

#show important features
display(xgb_feat_df)

#Make predictions on train and test set
y_train_pred = xgb_reg.predict(X_train)
y_test_pred = xgb_reg.predict(X_test)

#Calculate R² scores for train and test predictions
r2_train = r2_score(y_train,y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

#Compare R2 Score for the first 10 instances
vergleich_df = pd.DataFrame({
    'True value' : y_test[:10],
    'Prediction' : y_test_pred[:10].flatten(),
    'Difference' : y_test[:10] - y_test_pred[:10].flatten()
 })

#print the result
print(f'\nR² Train: {r2_train:.4f} | R² Test: {r2_test:.4f}')
display(vergleich_df.style.background_gradient(cmap='RdYlGn_r'))

#! Uncertainty for XGBOOST --> commin soon

,Feature,Importance
5,Ziehtiefe [mm],0.745549
1,Stempelkantenradius [mm],0.125797
3,Platinenabmessung [mm],0.123737
4,1/4 Niederhalterkraft [N],0.002886
0,Matrizenradius [mm ],0.001195
2,Blechdicke [mm],0.000837



R² Train: 1.0000 | R² Test: 0.9997


,True value,Prediction,Difference
0,6.590000,6.567898,0.022102
1,8.100000,8.085225,0.014775
2,9.231000,9.233231,-0.002231
3,7.623000,7.621459,0.001541
4,9.253500,9.222149,0.031351
5,10.484500,10.444893,0.039607
6,8.219000,8.217233,0.001767
7,6.679500,6.685080,-0.005580
8,8.614500,8.610810,0.003690
9,7.534500,7.529448,0.005052
